In [1]:
%reload_ext autoreload
%autoreload 2

In [12]:
from trader import Trader
from utils import id_token

size, delta = 15, 3
minutes = 2

alpha = 0.3

alphas = [0.1, 0.3, 0.5]
sizes = [5, 10, 15, 20, 25]
deltas = [0, 3 ,6, 9]
statistics = []

for delta in deltas:
    trader = Trader(id_token, size, alpha, delta, False)
    stats = trader.trade(minutes)
    statistics.append(stats)

In [2]:
from trader import Trader
from utils import id_token

# trader = Trader(id_token, 15, 0.3, 3, True)
# stats = trader.trade(1e3)

Profit: 3396.672 USD. Percentage profit: 0.113%
Overall profit: 3396.672 USD
Profit: 3571.351 USD. Percentage profit: 0.119%
Overall profit: 6968.023 USD
Profit: 3870.353 USD. Percentage profit: 0.129%
Overall profit: 10838.376 USD
Profit: 254.156 USD. Percentage profit: 0.008%
Overall profit: 11092.532 USD
Profit: 6652.300 USD. Percentage profit: 0.221%
Overall profit: 17744.832 USD
Profit: 1943.094 USD. Percentage profit: 0.064%
Overall profit: 19687.926 USD
Profit: -104.284 USD. Percentage profit: -0.003%
Overall profit: 19583.642 USD
Profit: -1477.610 USD. Percentage profit: -0.049%
Overall profit: 18106.032 USD
Profit: -188.134 USD. Percentage profit: -0.006%
Overall profit: 17917.898 USD
Profit: 2038.845 USD. Percentage profit: 0.067%
Overall profit: 19956.743 USD
Profit: 4536.186 USD. Percentage profit: 0.225%
Overall profit: 24492.929 USD
Profit: 3707.518 USD. Percentage profit: 0.122%
Overall profit: 28200.447 USD
Profit: 2682.185 USD. Percentage profit: 0.264%
Overall profit:

In [14]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

def plot_graph(stats):
    profits = np.array(stats['Profits'])
    all_profits = stats['All_profits']
    btc_price = np.array(stats['BTC_price'])
    btc_change = (btc_price-btc_price[0])/btc_price[0]
    profit = (profits[:,0]-profits[:,1])/profits[:,1]
    mean = np.mean(profit)
    std = np.std(profit, ddof=1)

    fig = px.line(x=profits[:,2], y=profit, title=f'Total profit = {all_profits:.3f}. Mean profit: {mean*100:.3f}%. Coef of variation: {std/mean*100:.3f}%')
    fig.add_trace(go.Scatter(y=btc_change, x=list(range(len(btc_price))), mode="lines", name="BTC Change"))

    fig.show()

for delta, stats in zip(deltas, statistics):
    if len(stats['Profits']) > 0:
        print(delta)
        plot_graph(stats)


0


In [5]:
from utils import id_token
from helper_functions import buy_btc, sell_btc, read_off_crypto_data, make_multiple_requests

# https://platform.the-brawl.eu/ui/api/rates?currencyId=BTC&type=current

session = requests.Session()

res = session.post(
        'https://platform.the-brawl.eu/ui/api/rates?currencyId=BTC&type=current',
        headers={"Authorization": f"Bearer {id_token}"},
        json={
            "currencyId": 'BTC',
            "type": 'current',
            # "amountFromSourceWallet": price * volume,
            # "exchangeRate": 1 / price,
        },
        verify=False,
    )
print(res.json())

{'ok': False, 'code': 'unauthorized'}


In [179]:
%timeit buy_btc(0.000001, 42300, id_token)

138 ms ± 12.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [152]:
%timeit sell_btc(0.000001, 41300, id_token)

241 ms ± 47.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [127]:
COINBASE_BTC_URL = "https://api.coinbase.com/v2/prices/BTC-USD/spot"
COINBASE_ETH_URL = "https://api.coinbase.com/v2/prices/ETH-USD/spot"
COINBASE_TIME_URL = "https://api.coinbase.com/v2/time"

urls = [
            COINBASE_TIME_URL,
            COINBASE_BTC_URL,
            COINBASE_ETH_URL,
]

In [135]:
%timeit read_off_crypto_data(make_multiple_requests(urls))

223 ms ± 39.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [160]:
import requests
from datetime import datetime
import numpy as np

with requests.Session() as session:
    start = datetime.now()
    response = session.get("https://api.coinbase.com/v2/prices/BTC-USD/spot")
    end = datetime.now()
    print((end-start).total_seconds())
    times = []
    for _ in range(100):
        start = datetime.now()
        session.get("https://api.coinbase.com/v2/prices/BTC-USD/spot")
        end = datetime.now()
        times.append((end-start).total_seconds())
    print(np.mean(times), np.var(times))


0.196125
0.19561863 0.0018980419689130998


In [167]:
import os

TRANSACTION_URL = "http://platform.the-brawl.eu/api/transaction"
BTC_WALLET_ID = os.environ.get("BTC_WALLET_ID")
USD_WALLET_ID = os.environ.get("USD_WALLET_ID")

def buy_btc(volume, price, token, session):
    # print(f"Buying {volume} BTC for {price} per unit USD, total: {volume * price} USD")
    res = session.post(
        TRANSACTION_URL,
        headers={"Authorization": f"Bearer {token}"},
        json={
            "sourceWalletId": USD_WALLET_ID,
            "destWalletId": BTC_WALLET_ID,
            "amountFromSourceWallet": price * volume,
            "exchangeRate": 1 / price,
        },
        verify=False,
    )
    return "data" in res.json().keys()

from concurrent.futures import ThreadPoolExecutor
import json

def make_request(url):
    response = session_coinbase.get(url)
    return response.text

session_coinbase = requests.Session()

def make_multiple_requests(urls, max_workers=3):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        responses = list(executor.map(make_request, urls))

    data = {}
    for url, response in zip(urls, responses):
        data[url] = json.loads(response)

    return data

In [170]:
volume = 0.00001
price = 1
token = id_token


start = datetime.now()
response = make_multiple_requests(urls)
end = datetime.now()
print((end-start).total_seconds())
times = []
for _ in range(100):
    start = datetime.now()
    make_multiple_requests(urls)
    end = datetime.now()
    times.append((end-start).total_seconds())
print(np.mean(times), np.var(times))

0.193977
0.19554860999999998 0.0033851800912578995


In [166]:
volume = 0.00001
price = 1
token = id_token

with requests.Session() as session:
    start = datetime.now()
    response = buy_btc(volume, price, token, session)
    end = datetime.now()
    print((end-start).total_seconds())
    times = []
    for _ in range(100):
        start = datetime.now()
        buy_btc(volume, price, token, session)
        end = datetime.now()
        times.append((end-start).total_seconds())
    print(np.mean(times), np.var(times))

0.184947
0.08321361 0.0008180979127179001


In [185]:
headers = {
    "authority": "platform.the-brawl.eu",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br",
    "accept-language": "en-GB,en-US;q=0.9,en;q=0.8,pl;q=0.7",
    "cache-control": "max-age=0",
    "cookie": "_ga=GA1.1.1273563410.1702031817; connect.sid=s%3Aj7CTFkw3WubU7VoMUphEKdZccBXhqljY.onbbx7GdAKHqWRowPJWavbLkL%2Fp4eQlJoF1Cd9p%2B3kw; cryptobrawl.AuthUser=IntcImlkXCI6XCJHbGpiakE4a1dqV2lHS2FrUEV2S3VlQjZHODEyXCIsXCJjbGFpbXNcIjp7fSxcImVtYWlsXCI6XCJwbjQ1ODg2NkBzdHVkZW50cy5taW11dy5lZHUucGxcIixcImVtYWlsVmVyaWZpZWRcIjp0cnVlLFwiZGlzcGxheU5hbWVcIjpcIlBhd2VsXCIsXCJjbGllbnRJbml0aWFsaXplZFwiOmZhbHNlfSI=; cryptobrawl.AuthUser.sig=prGKP_opFy9YWZZc3i0bTgTwE_Y; _ga_85WP5H5EX2=GS1.1.1702626284.38.1.1702627248.0.0.0; cryptobrawl.AuthUserTokens=IntcImlkVG9rZW5cIjpcImV5SmhiR2NpT2lKU1V6STFOaUlzSW10cFpDSTZJbUpsTnpneU0yVm1NREZpWkRSa01tSTVOakkzTkRFMk5UaGtNakE0TURkbFptVmxObVJsTldNaUxDSjBlWEFpT2lKS1YxUWlmUS5leUp1WVcxbElqb2lVR0YzWld3aUxDSnBjM01pT2lKb2RIUndjem92TDNObFkzVnlaWFJ2YTJWdUxtZHZiMmRzWlM1amIyMHZZM0o1Y0hSdk1qTXROV0ZsTVRFaUxDSmhkV1FpT2lKamNubHdkRzh5TXkwMVlXVXhNU0lzSW1GMWRHaGZkR2x0WlNJNk1UY3dNall5TnpJME9Dd2lkWE5sY2w5cFpDSTZJa2RzYW1KcVFUaHJWMnBYYVVkTFlXdFFSWFpMZFdWQ05rYzRNVElpTENKemRXSWlPaUpIYkdwaWFrRTRhMWRxVjJsSFMyRnJVRVYyUzNWbFFqWkhPREV5SWl3aWFXRjBJam94TnpBeU5qSTNNalE0TENKbGVIQWlPakUzTURJMk16QTRORGdzSW1WdFlXbHNJam9pY0c0ME5UZzROalpBYzNSMVpHVnVkSE11YldsdGRYY3VaV1IxTG5Cc0lpd2laVzFoYVd4ZmRtVnlhV1pwWldRaU9uUnlkV1VzSW1acGNtVmlZWE5sSWpwN0ltbGtaVzUwYVhScFpYTWlPbnNpWlcxaGFXd2lPbHNpY0c0ME5UZzROalpBYzNSMVpHVnVkSE11YldsdGRYY3VaV1IxTG5Cc0lsMTlMQ0p6YVdkdVgybHVYM0J5YjNacFpHVnlJam9pWTNWemRHOXRJbjE5LmVMWlNKNzJnd05TSVBVSlQ0SU40bmRJdWgtM3V6T2xBSjMtYV9MVHlUV0hpUWM3YzFDcUJ6ZWN5akc1dXNYRlR4MnlkdkV6RFRyTHBJTHZMamEyTWYycFdnR05ON1IzUGtPeEQyVHNxQ0JjVDNzRExhOXc1b19MQ3Z6dGFHMk5vbE9LNEo0ZG9hamxqQi15V1FGSEo2T2dQTmVDUkIxQTF3ejhpd3hjV1p1LW1mcmgtSEpPSmREY1pnMVl4N0RUYmgyTHVCUHpyMXdLSEh3eVIySEJBeWFEVFUyUXdQU3pzY3BQczhjU2FjN2d6Y3AxbndUV0lCWjN0YXBGbkR5LXFCRU9BenE2SUdLdGplaFIyQUVtaWR2aGdKLXRPRWExYlUtR0FKai1fakJQdjVoYWxnR2ZDTVducXV0ZG5NUHdWUWlRRkI5LXYxQ0s3S1RLakFxOTNhQVwiLFwicmVmcmVzaFRva2VuXCI6XCJBTWYtdkJ5SWM0RWZ1OVVPWjBTb0VJb1JxbVZyc0JxWmtUZHNRM2FQNDZ2Y3FIUk1HUl9sUkg5bWtkYXJJVXQ3ZWpXcjZBRlJnTkZGejFQZlVfUWJuTldxZDYxQWdsZ0s0VWliaWVEY1Q1emVDQjVPcFc1NFFvNER0N3ljbkJhODUwcFRoMEgtbnhvU1M3REN2aFZxY3pkX205eXpRZnR0Qm5aUDczTEhhZllTTG14dWZqSlRhVjhcIn0i; cryptobrawl.AuthUserTokens.sig=W23CqTQHywU2T54jxaB3Vc6dsrI",
    "if-none-match": '"24a-KVY3R5JRe8if66f102/e+gBdqRI"',
    "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "sec-gpc": "1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}
url = "https://platform.the-brawl.eu/ui/api/rates?currencyId=BTC&type=current"

r = requests.Session()

responsee = r.get(url, headers=headers, verify=False)